# KDNuggets Blockchain in 7 Steps Blogpost

https://www.kdnuggets.com/2018/04/blockchain-explained-7-python-functions.html

In [2]:
import json
import hashlib

> At the heart of the blockchain is the hashing function

In [21]:
def hash_function(k):
    """Hashes our transaction"""
    if type(k) is not str:
        k = json.dumps(k, sort_keys=True)
        k = k.encode('utf-8')
        
    return hashlib.sha256(k).hexdigest()

> The ‘state’ is the record of who owns want. For example, I have 10 coins and I give 1 to Medium, then the state will be the value of the dictionary below.

`{‘transaction’: {‘Tom’: 9, ‘Medium’: 1}}`

### Function 1.  Update State

In [13]:
def update_state(transaction, state):
    state = state.copy()
    
    for key in transaction:
        if key in state.keys():
            state[key] += transaction[key]
        else:
            state[key] = transaction[key]
            
    return state

> ..overdrafts cannot exist.. transactions must balance

### Function 2. Validate Transaction

In [14]:
def valid_transaction(transaction, state):
    """A valid transaction must sum to 0"""
    if sum(transaction.values()) is not 0:
        return False
    
    for key in transaction.keys():
        if key in state.keys():
            account_balance = state[key]
        else:
            account_balance = 0
            
        if account_balance + transaction[key] < 0:
            return False
        
    return True

Build the block

### Function 3.  Make Block

In [32]:
def make_block(transactions, chain):
    """Make a block to go into the chain."""
    parent_hash = chain[-1]['hash']
    block_number= chain[-1]['contents']['block_number'] + 1
    
    block_contents = {
        'block_number': block_number,
        'parent_hash': parent_hash,
        'transaction_count': block_number + 1,
        'transaction': [transactions]
    }
    
    return {'hash': hash_function(block_contents),
           'contents': block_contents }

Check contents of previous block

### Function 4.  Check Block Hash

In [16]:
def check_block_hash(block):
    expected_hash = hash_function(block['contents'])
    
    if block['hash'] is not expected_hash:
        raise
        
    return

Create Block and Update Blockchain

### Function 5.  Check Block Validity

In [17]:
def check_block_validity(block, parent, state):
    parent_number = parents['contents']['block_number']
    parent_hash = parent['hash']
    block_number= block['contents']['block_number']
    
    for transaction in block['contents']['transaction']:
        if valid_transaction(transaction, state):
            state = update_state(transaction, state)
        else:
            raise
            
    check_block_hash(block)
    
    if block_number is not parent_number + 1:
        raise
        
    if block['contents']['parent_hash'] is not parent_hash:
        raise
        
    return state

Verify and Check Chain

### Function 6.  Check Chain

In [18]:
def check_chain(chain):
    """Check if chain is valid"""
    if type(chain) is str:
        try:
            chain = json.loads(chain)
            assert (type(chain) == list)
        except ValueError:
            # String passed in was not valid JSON
            return False
    elif type(chain) is not list:
        return False
    
    state = {}
    
    for transaction in chain[0]['contents']['transaction']:
        state = update_state(transaction, state)
        
    check_block_hash(chain[0])
    parent = chain[0]
    
    for block in chain[1:]:
        state = check_block_validity(block, parent, state)
        parent= block
        
    return state

Transaction function 

### Function 7.  Transaction to Chain

In [19]:
def add_transaction_to_chain(transaction, state, chain):
    if valid_transaction(transaction, state):
        state = update_state(transaction, state)
    else:
        raise Exception('Invalid transaction.')
        
    my_block = make_block(state, chain)
    chain.append(my_block)
    
    for transaction in chain:
        check_chain(transaction)
        
    return state, chain

## blockchain data

In [26]:
genesis_block = {
    'hash': hash_function({
        'block_number': 0,
        'parent_hash': None,
        'transaction_count': 1,
        'transaction': [{'Tom': 10}]
    }),
    'contents': {
        'block_number': 0,
        'parent_hash': None,
        'transaction_count': 1,
        'transaction': [{'Tom': 10}]  
    }
}

block_chain = [genesis_block]
chain_state = {'Tom': 10}

## test transaction

In [33]:
chain_state, block_chain = add_transaction_to_chain(transaction=
                                                   {'Tom': -1,
                                                    'Medium': 1},
                                                   state=chain_state,
                                                   chain=block_chain)

In [34]:
block_chain, chain_state

([{'contents': {'block_number': 0,
    'parent_hash': None,
    'transaction': [{'Tom': 10}],
    'transaction_count': 1},
   'hash': '064d0b480b3b92761f31831d30ae9f01954efaa62371b4b44f11465ec22abe93'},
  {'contents': {'block_number': 1,
    'parent_hash': '064d0b480b3b92761f31831d30ae9f01954efaa62371b4b44f11465ec22abe93',
    'transaction': [{'Medium': 1, 'Tom': 9}],
    'transaction_count': 2},
   'hash': '3003c0039b4a9c9d13f028c3572567864746f40dcaec8b000ae6ca562c56409e'}],
 {'Medium': 1, 'Tom': 9})